In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Embedding
import tensorflow_addons as tfa
from tensorflow_addons.seq2seq import AttentionWrapper


We first load the preprocessed data.

In [4]:
data = np.load('../preprocessing/preprocessed.npz')
x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']
max_text_len = data['max_text_len']
max_summary_len = data['max_summary_len']
x_voc_size = data['x_voc_size']
y_voc_size = data['y_voc_size']

In this notebook we build up the Encoder-Decoder architecture using LSTM model. 

In [6]:
hidden_size = 1000


encoder_input = Input(shape=(max_text_len, ))
enc_emb = Embedding(input_dim=x_voc_size, output_dim=hidden_size)(encoder_input)

encoder = LSTM(hidden_size, return_sequences=True, return_state=True)
encoder_output, state_h, state_c = encoder(enc_emb)

decoder_input = Input(shape=(None, ))
dec_emb = Embedding(input_dim = y_voc_size, output_dim=hidden_size)(decoder_input)

decoder = LSTM(hidden_size, return_sequences=True, return_state=True)
decoder_output, state_h, state_c = decoder(dec_emb, initial_state=[state_h, state_c])

#attention = AttentionWrapper()

model = Model([encoder_input, decoder_input], decoder_output)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5000)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 5000, 1000)   14535000    ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 1000)   14535000    ['input_4[0][0]']                
                                                                                              